# Imports

In [1]:
from utils.nfl_data import (get_weekly_df, get_seasonal_df, get_roster_df, clean_df_for_csv,
                            create_mappings, create_position_groups, create_positional_df)

# Setting Years List

The years past into this list provides us with the data in each of those seasons.

**TODO:** need a function that can generate a year or years data for each position. This should then be used for github actions to update weekly stats. Additionally, can be used to retroactively update previous years if data changes.

In [2]:
year = 2023
years = [year]

# Seasonal and Weekly Data

In [3]:
df_weekly = get_weekly_df(years=years, columns=[])

Downcasting floats.


In [4]:
df_seasonal = get_seasonal_df(years=years)

# Roster ID

For player mapping creations

In [5]:
columns = ["player_name", "player_id", "position"]
df_roster = get_roster_df(years=years, columns=columns)

# Create Player Mappings

In [6]:
player_map, id_map = create_mappings(df_roster=df_roster, df_seasonal=df_seasonal, df_weekly=df_weekly)

## Mapping Checks

In [7]:
# player_name = "Christian Watson"

In [8]:
# player_id = id_map[player_name]

In [9]:
# player = player_map[player_id]

In [10]:
# player["seasonal_df"]

In [11]:
# player["week_dfs"][-1]

# Creating Position Groups

In [12]:
qbs, rbs, wrs, tes = create_position_groups(player_map=player_map)

In [13]:
df_qb = create_positional_df(qbs)
df_rb = create_positional_df(rbs)
df_wr = create_positional_df(wrs)
df_te = create_positional_df(tes)

In [14]:
df_qb.head(1)

,Index,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,2,00-0026498,2023,REG,183,309,2260.0,9,8.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.162


In [15]:
df_rb.head(1)

,Index,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,17,00-0029892,2023,REG,0,0,0.0,0,0.0,0.0,...,0.031,0.098,0.035,0.154,0.048,0.062,0.095,0.059,0.325,0.039


In [16]:
df_wr.head(1)

,Index,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,6,00-0027944,2023,REG,0,0,0.0,0,0.0,0.0,...,0.031,0.102,0.021,0.167,0.03,0.051,0.094,0.05,0.176,0.041


In [17]:
df_te.head(1)

,Index,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,1,00-0024243,2023,REG,0,0,0.0,0,0.0,0.0,...,0.076,0.066,0.036,0.0,0.027,0.023,0.018,0.029,0.276,0.016


# Saving Data to CSV for Graphing Tools

## Quarterback Save

In [18]:
df_to_save_qb = clean_df_for_csv(df=df_qb, player_map=player_map)
df_to_save_qb.head(1)

,name,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,...,wopr_x per game,special_teams_tds per game,fantasy_points per game,fantasy_points_ppr per game,tgt_sh per game,wopr_y per game,dom per game,w8dom per game,yptmpa per game,total_epa per game
0,Matthew Stafford,183,309,2260.0,9,8.0,20.0,150.0,0,0,...,0.0,0.0,13.477778,13.477778,0.0,0.0,0.0,0.0,0.0,1.538667


In [19]:
df_to_save_qb.to_csv(f"FantasyData/advanced-stats/QB/{year}.csv")

## Running Back Save

In [20]:
df_to_save_rb = clean_df_for_csv(df=df_rb, player_map=player_map)
df_to_save_rb.head(1)

,name,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,...,wopr_x per game,special_teams_tds per game,fantasy_points per game,fantasy_points_ppr per game,tgt_sh per game,wopr_y per game,dom per game,w8dom per game,yptmpa per game,total_epa per game
0,Kyle Juszczyk,0,0,0.0,0,0.0,0.0,0.0,0,0,...,0.107,0.0,2.728571,3.871429,0.007143,0.014,0.013571,0.008429,0.046429,-0.272429


In [21]:
df_to_save_rb.to_csv(f"FantasyData/advanced-stats/RB/{year}.csv")

# Wide Receiver Save

In [22]:
df_to_save_wr = clean_df_for_csv(df=df_wr, player_map=player_map)
df_to_save_wr.head(1)

,name,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,...,wopr_x per game,special_teams_tds per game,fantasy_points per game,fantasy_points_ppr per game,tgt_sh per game,wopr_y per game,dom per game,w8dom per game,yptmpa per game,total_epa per game
0,Julio Jones,0,0,0.0,0,0.0,0.0,0.0,0,0,...,0.105,0.0,2.533333,3.866667,0.018333,0.034,0.031333,0.016667,0.058667,0.0


In [23]:
df_to_save_wr.to_csv(f"FantasyData/advanced-stats/WR/{year}.csv")

# Tight End Save

In [24]:
df_to_save_te = clean_df_for_csv(df=df_te, player_map=player_map)
df_to_save_te.head(1)

,name,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,...,wopr_x per game,special_teams_tds per game,fantasy_points per game,fantasy_points_ppr per game,tgt_sh per game,wopr_y per game,dom per game,w8dom per game,yptmpa per game,total_epa per game
0,Marcedes Lewis,0,0,0.0,0,0.0,0.0,0.0,0,0,...,0.083333,0.0,0.8,1.466667,0.011333,0.022,0.006,0.009667,0.092,0.0


In [25]:
df_to_save_te.to_csv(f"FantasyData/advanced-stats/TE/{year}.csv")